This Jupyter file is designed to conduct an in-depth analysis of the dataset, incorporating statistical insights and visualization. Throughout the file, we show various visual representations and numerical summaries. The final step involves saving the modified dataset in CSV format for further utilization.

# Importing Libraries 

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Analysis of BPI_Challenge_2012 data set

In [ ]:
# Loading the data
df = pd.read_csv("data/preprocessed/BPI_Challenge_2012.csv")
df.head(10)

### Short Summary of Data (All the datasets)

In [ ]:
# Printing the nr rows and columns
print(f'nr Rows: {df.shape[0]}, nr Cols: {df.shape[1]}\n')

# checking the data types
print(df.info())

### Missing Values

In [ ]:
# checking the missing values
df.isnull().sum().tail(40)

In [ ]:
# checking the percentage of missing values for each column
missing = df.isnull().sum()
missing = missing[missing > 0]
missing = missing / df.shape[0] * 100
missing

Analysis of unique values for each attribute

In [ ]:
# checking the unique values
for col in df.columns:
    print(f'{col}: {df[col].nunique()}')

In [ ]:
# checking the unique values for the 'concept:name' column
df['concept:name'].value_counts()

In [ ]:
# checking the unique values of the lifecycle:transition
df['lifecycle:transition'].unique()

In [ ]:
# Plotting the freq of the lifecycle:transition values
plt.figure(figsize=(10, 5))
df['lifecycle:transition'].value_counts().plot(kind='bar')
plt.title('Freq of lifecycle:transition')   
plt.xlabel('lifecycle:transition')
plt.ylabel('Frequency')
plt.xticks(rotation=0)
plt.savefig('figs/freq_lifecycle_transition.png')
plt.show()

print(df['lifecycle:transition'].value_counts())

In [ ]:
# Plotting the freq of the concept:name values
plt.figure(figsize=(20, 10))
df['concept:name'].value_counts().plot(kind='bar')
for i, v in enumerate(df['concept:name'].value_counts()):
    plt.text(i, v + 0.2, str(v), ha='center', va='bottom')
plt.xlabel('concept:name', fontsize=15)
plt.ylabel('Frequency', fontsize=15)
plt.xticks(rotation=90)
plt.title('Freq of concept:name', fontsize=15)  
# increase the font size
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)

plt.tight_layout()
plt.savefig('figs/freq_concept_name.png')
plt.show()

In [ ]:
# checking the unique values for the case:concept:name column
df['case:concept:name'].value_counts()

In [ ]:
position_dict = {}
for j in range(1, max(df["position"]+1)):
    dic = {}
    for i in (df[df['position'] == j]).index:
        if df['concept:name'][i] in dic:
            dic[df['concept:name'][i]] += 1
        else:
            dic[df['concept:name'][i]] = 1
    
    position_dict[j] = max(dic, key=dic.get)

nr_positions_dominant_in = {}

for i in position_dict.items():
    if i[1] in nr_positions_dominant_in:
        nr_positions_dominant_in[i[1]] += 1
    else:
        nr_positions_dominant_in[i[1]] = 1

# printing the dominant activity in each position in percentage
for i in nr_positions_dominant_in.items():
    print(f'{i[0]}: {i[1]/max(df["position"])*100:.2f}%')

In [ ]:
del df

# Analysis specific to the BPI_Challenge_2018 dataset

In [ ]:
df = pd.read_csv("data/preprocessed/BPI_Challenge_2018.csv")

In [ ]:
# checking the percentage of missing values for each column
missing = df.isnull().sum()
missing = missing[missing > 0]
missing = missing / df.shape[0] * 100
missing

In [ ]:
# Finding all the columns that have values either True or False
bool_cols = [col for col in df.columns if np.isin(df[col].dropna().unique(), [True, False]).all()]
bool_cols, len(bool_cols)

In [ ]:
# Removing the boolean_columns from the dataframe
df_no_bool = df.drop(columns=bool_cols)
df_no_bool.head(5)

In [ ]:
df_no_bool.columns

In [ ]:
# finding the rows that have the value '0;n/a' in the 'org:resource' column since isna() does not work for this value
df_no_bool[df_no_bool['org:resource'] == '0;n/a']

In [ ]:
# Since the isna() method does not capture 0;n/a as missing values, we need to replace it with np.nan
df_no_bool['org:resource'] = df_no_bool['org:resource'].replace('0;n/a', np.nan)
df_no_bool['org:resource'].isna().sum()/len(df_no_bool) * 100

In [ ]:
# Checking the value counts for the subprocess column
df_no_bool['subprocess'].value_counts()

In [ ]:
# Plotting the freq of the subprocess values
plt.figure(figsize=(20, 10))
df_no_bool['subprocess'].value_counts().plot(kind='bar')
for i, v in enumerate(df_no_bool['subprocess'].value_counts()):
    plt.text(i, v + 0.2, str(v), ha='center', va='bottom')
plt.title('Freq of subprocess')
plt.show()

## Analysis of the boolean columns

In [ ]:
# A dataframe with only the boolean columns
df_bool = df[bool_cols]

# checking for the number of unique values for each column
unique = df_bool.nunique()
unique

In [ ]:
# drpoping the case:greening and case:basic payment columns since they have only one unique value
df_bool = df_bool.drop(columns=['case:greening', 'case:basic payment'])

In [ ]:
# Finding the similarity between the boolean columns
similarity = df_bool.corr()
similarity

In [ ]:
# Finding the columns that have a correlation above 0.7 and they similarity
corr_cols = []
for i in range(similarity.shape[0]):
    for j in range(i+1, similarity.shape[0]):
        if abs(similarity.iloc[i, j]) > 0.5:
            corr_cols.append((similarity.columns[i], similarity.columns[j], similarity.iloc[i, j] ))
corr_cols

In [ ]:
# Plotting the similarity matrix
plt.figure(figsize=(20, 10))
plt.matshow(similarity, cmap='coolwarm')
plt.colorbar()
plt.title('Similarity Matrix')
plt.show()

In [ ]:
df_no_bool['case:amount_applied3'].isna().sum()

In [ ]:
print(df_no_bool['case:risk_factor'].value_counts() / len(df_no_bool) * 100)

# Dropping the risk factor column since it does not contain much information
df_no_bool = df_no_bool.drop(columns=['case:risk_factor'])

In [ ]:
print(df_no_bool['case:cross_compliance'].value_counts()/len(df_no_bool) * 100)

# Dropping the cross_compliance column since it does not contain much information
df_no_bool = df_no_bool.drop(columns=['case:cross_compliance'])

In [ ]:
print(df['case:program-id'].value_counts() / len(df) * 100)

# Dropping the program-id column since it does not contain much information
df_no_bool = df_no_bool.drop(columns=['case:program-id'])

In [ ]:
dropping_columns = ['case:penalty_amount1', 'case:penalty_amount2', 'case:penalty_amount3', 'case:payment_actual1',
                    'case:payment_actual2', 'case:payment_actual3', 'case:amount_applied1', 'case:amount_applied2', 'case:amount_applied3']

# Dropping the above columns since they do not contain much information due to the high percentage of missing values
df_no_bool = df_no_bool.drop(columns=dropping_columns)

In [ ]:
# checking what columns are constant across the same case:concept:name
df_no_bool.groupby('case:concept:name').nunique().nunique()

In [ ]:
# correlation between the columns case:amount_applied0 and case:amount_actual0
print(df_no_bool[['case:amount_applied0', 'case:payment_actual0']].corr())

# dropping one of the columns since they are highly correlated
df_no_bool = df_no_bool.drop(columns=['case:payment_actual0'])

In [ ]:
# dropping the boolean columns that are highly correlated with other columns
df_bool = df_bool.drop(columns=['case:penalty_AVBP', 'case:penalty_AGP', 'case:selected_random'])

In [ ]:
# Final columns to be used for the further steps
df_no_bool.columns, df_bool.columns